In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from io import BytesIO
from pptx import Presentation
from pptx.util import Inches

# 데이터 프레임 생성
np.random.seed(0)
data = {
    'cat1': np.random.choice(['ProductA', 'ProductB', 'ProductC', 'ProductD', 'ProductE', 'ProductF', 'ProductG'], 3000),
    'cat2': np.random.choice(['Setting1', 'Setting2', 'Setting3', 'Setting4', 'Setting5', 'Setting6', 'Setting7', 'Setting8'], 3000),
    'cat3': np.random.randint(1, 11, 3000),
    'x': np.random.randint(1, 11, 3000),
    'y': np.random.randint(1, 11, 3000),
    'datetime': pd.date_range(start='2021-01-01', periods=3000, freq='D')
}
df = pd.DataFrame(data)
vals = {f'val{val_num}': np.random.random(3000) for val_num in range(1, 70)}
df = pd.concat([df, pd.DataFrame(vals)], axis=1)

# 'cat1', 'cat2', 'cat3', 'x', 'y', 'datetime' 열을 제외한 리스트 구하기
excluded_columns = ['cat1', 'cat2', 'cat3', 'x', 'y', 'datetime']
timeseries_columns = [col for col in df.columns if col not in excluded_columns]

# PPTX 프레젠테이션 생성
prs = Presentation()

# 각 25개 열에 대해 이미지 생성 및 PPTX에 삽입
for i in range(0, len(timeseries_columns), 25):
    current_columns = timeseries_columns[i:i+25]
    fig, axs = plt.subplots(5, 5, figsize=(15, 15), dpi=100)  # 5x5 그리드 생성
    
    # 현재 그룹의 각 열에 대해 플롯 생성
    for j, column in enumerate(current_columns):
        ax = axs.flatten()[j]
        ax.scatter(df.index, df[column], alpha=0.5)
        ax.set_title(column, fontsize=12)

    # 나머지 빈칸은 비워둠
    for k in range(j + 1, 25):
        axs.flatten()[k].axis('off')

    plt.tight_layout()
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)

    # 슬라이드 추가 및 이미지 삽입
    slide_layout = prs.slide_layouts[6]  # 가장 기본적인 레이아웃
    slide = prs.slides.add_slide(slide_layout)
    slide.shapes.add_picture(buf, Inches(0), Inches(0), width=prs.slide_width, height=prs.slide_height)

# PPTX 파일 저장
pptx_file_path = "timeseries_plots_combined.pptx"
prs.save(pptx_file_path)
print(f"PPTX 파일이 저장된 위치: {pptx_file_path}")

PPTX 파일이 저장된 위치: timeseries_plots_combined.pptx
